In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math

In [2]:
def generate_vectormap(xc, yc, intenses, rect=[[-5, -5], [5, 5]], eps=1.0):
    x = np.linspace(rect[0][0], rect[1][0], int((rect[1][0] - rect[0][0])/eps))
    y = np.linspace(rect[0][1], rect[1][1], int((rect[1][1] - rect[0][1])/eps))
    x, y = np.meshgrid(x, y)
    
    def E(g, a, b, x, y, delta=0.1):        
        R = np.maximum(np.abs(((x-float(a))**2+(y-float(b))**2)), delta)
        v = (float(g)/2.0*math.pi) * ((x-a)/R)
        u = (float(g)/2.0*math.pi) * ((b-y)/R)
        return u, v
    
    xc = np.array(xc, dtype=np.float_)
    yc = np.array(yc, dtype=np.float_)
    assert(xc.shape == yc.shape)
    
    Ex, Ey = E(intenses[0], xc[0], yc[0], x, y)
    for i in range(1, xc.shape[0]):
        Ex_, Ey_ = E(intenses[i], xc[i], yc[i], x, y)
        Ex = Ex + Ex_
        Ey = Ey + Ey_
     
    return x, y, Ex, Ey
    #plt.quiver(x, y, Ex/np.sqrt(Ex**2+Ey**2), Ey/np.sqrt(Ex**2+Ey**2), pivot='middle', 
    #           headwidth=3, headlength=6, color='gray')
    #plt.plot(xc, yc, 'ro')
#    #plt.show()

In [3]:


xc = [0]
yc = [0]
intenses = [1]

#xc = [-1, -1, 1, 1, 0]
#yc = [-1, 1, -1, 1, 0]
#intenses = [1/5, 1/5, 1/5, 1/5, 1/5]

x, y, Ex, Ey = generate_vectormap(xc, yc, intenses, rect=[[-2, -2], [2, 2]], eps=0.1)

plt.quiver(x, y, Ex/np.sqrt(Ex**2+Ey**2), Ey/np.sqrt(Ex**2+Ey**2), pivot='middle', 
           headwidth=4, headlength=6, color='gray')
plt.plot(xc, yc, 'ro')
plt.show()

x_init = 0.5
y_init = 0.5

In [5]:
def V(x, y, xc, yc, g):
    def E(g, a, b, x, y, delta=0.01):        
        R = np.maximum(np.abs(((x-float(a))**2+(y-float(b))**2)), delta)
        v = (float(g)/2.0*math.pi) * ((x-a)/R)
        u = (float(g)/2.0*math.pi) * ((b-y)/R)
        return u, v
    
    assert len(xc) == len(yc) == len(g)
    u, v = 0, 0
    for i in range(len(xc)):
        du, dv = E(g[i], xc[i], yc[i], x, y)
        u = u + du
        v = v + dv
        
    return u, v

def R(x, y, xprev, yprev, dt, xc, yc, g):
    assert len(xc) == len(yc) == len(g)    
    
    uprev, vprev = V(xprev, yprev, xc, yc, g)
    u, v = V(x, y, xc, yc, g)
    
    x = x + dt/2 * (u + uprev)
    y = y + dt/2 * (v + vprev)
    
    return x, y

def euler_draw_path(x0, y0, u0, v0, xc, yc, intenses, rect):
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'g-', lw=2)
    point, = ax.plot(x0, y0, 'bo')
    ax.set_ylim(rect[0][1], rect[1][1])
    ax.set_xlim(rect[0][0], rect[1][0])
    
    tdata = [0]
    xdata = [x0 - u0, x0]
    ydata = [y0 - v0, y0]
    udata = [0, u0]
    vdata = [0, v0]
    
    def run(t):
        # update the data            
        i = len(xdata)
        x, y = R(xdata[len(xdata)-1], ydata[len(ydata)-1], 
                 xdata[len(xdata)-2], ydata[len(ydata)-2], t-tdata[len(tdata)-1], xc, yc, intenses)
        
        tdata.append(t)
        xdata.append(x)
        ydata.append(y)
                
        point.set_data(x, y)
        line.set_data(xdata[1:], ydata[1:])
        return line, point,
    
    
    x, y, Ex, Ey = generate_vectormap(xc, yc, intenses, rect, eps=0.1)
    
    ax.quiver(x, y, Ex/np.sqrt(Ex**2+Ey**2), Ey/np.sqrt(Ex**2+Ey**2), pivot='middle', 
               headwidth=4, headlength=6, color='gray')
    ax.plot(xc, yc, 'ro')
    ani = animation.FuncAnimation(fig, run, np.arange(0, 100, 0.005), blit=True, interval=0.001,
        repeat=False)
    plt.show()

In [6]:
xc = [-1.0, -1.0, 1.0, 1.0, 0.0]
yc = [-1.0, 1.0, -1.0, 1.0, 0.0]
intenses = [1/5, 1/5, 1/5, 1/5, 1/5]

#xc = [-1.0, -0.5, 1.0, 0.5, 0.0]
#yc = [0, 0, 0, 0, 0]

#Masha:
#xc = [-0.5, -0.5, 0.5, 0.5]
#yc = [-0.5, 0.5, -0.5, 0.5]
#intenses = [1/4, 1/4, 1/4, 1/4]

x0 = 0.5
y0 = 0.7
u0 = 0
v0 = 0

rect = [[-2, -2],[2, 2]]
euler_draw_path(x0, y0, u0, v0, xc, yc, intenses, rect)


In [17]:
def RK4(f):
    return lambda t, y, dt: (
            lambda dy1: (
            lambda dy2: (
            lambda dy3: (
            lambda dy4: (dy1 + 2*dy2 + 2*dy3 + dy4)/6
            )( dt * f( t + dt  , y + dy3   ) )
            )( dt * f( t + dt/2, y + dy2/2 ) )
            )( dt * f( t + dt/2, y + dy1/2 ) )
            )( dt * f( t       , y         ) )
 
def theory(t): return (t**2 + 4)**2 /16
 
from math import sqrt
dy = RK4(lambda t, y: t*sqrt(y))
 
t, y, dt = 0., 1., .1
while t <= 10:
    if abs(round(t) - t) < 1e-5:
        print("y(%2.1f)\t= %4.6f \t error: %4.6g" % ( t, y, abs(y - theory(t))))
    t, y = t + dt, y + dy( t, y, dt )
 

y(0.0)	= 1.000000 	 error:    0
y(1.0)	= 1.562500 	 error: 1.45722e-07
y(2.0)	= 3.999999 	 error: 9.19479e-07
y(3.0)	= 10.562497 	 error: 2.90956e-06
y(4.0)	= 24.999994 	 error: 6.23491e-06
y(5.0)	= 52.562489 	 error: 1.08197e-05
y(6.0)	= 99.999983 	 error: 1.65946e-05
y(7.0)	= 175.562476 	 error: 2.35177e-05
y(8.0)	= 288.999968 	 error: 3.15652e-05
y(9.0)	= 451.562459 	 error: 4.07232e-05
y(10.0)	= 675.999949 	 error: 5.09833e-05


In [7]:
def rk4_draw_path(x0, y0, dt, T, xc, yc, gc, rect):
    def RK4(f):
        return lambda t, y, dt, xc, yc, gc: (
                lambda dy1: (
                lambda dy2: (
                lambda dy3: (
                lambda dy4: (dy1 + 2*dy2 + 2*dy3 + dy4)/6
                )( dt * f( t + dt  , y + dy3   , xc, yc, gc) )
                )( dt * f( t + dt/2, y + dy2/2 , xc, yc, gc) )
                )( dt * f( t + dt/2, y + dy1/2 , xc, yc, gc) )
                )( dt * f( t       , y         , xc, yc, gc) )
 
    def V(x, y, xc, yc, gc):
        def E(g, a, b, x, y, delta=0.001):
            R = np.maximum(np.abs(((x-float(a))**2+(y-float(b))**2)), delta)
            v = (float(g)/2.0*math.pi) * ((x-a)/R)
            u = (float(g)/2.0*math.pi) * ((b-y)/R)
            return u, v
    
        assert len(xc) == len(yc) == len(gc)
        u, v = 0, 0
        for i in range(len(xc)):
            du, dv = E(gc[i], xc[i], yc[i], x, y)
            u = u + du
            v = v + dv        
        return np.array([u, v])
        
    dy = RK4(lambda t, y, xc, yc, gc: V(y[0], y[1], xc, yc, gc))
 
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'g-', lw=2)
    point, = ax.plot(x0, y0, 'bo')
    ax.set_ylim(rect[0][1], rect[1][1])
    ax.set_xlim(rect[0][0], rect[1][0])
    
    tprev = 0.0
    xdata = [x0]
    ydata = [y0]
    
    def run(t):
        # update the data            
        
        v = np.array([xdata[len(xdata)-1], ydata[len(ydata)-1]])
        #dt = t - tprev
        
        v = v + dy(t, v, dt, xc, yc, gc)
        
        #tprev = t
        xdata.append(v[0])
        ydata.append(v[1])
        
        point.set_data(v[0], v[1])
        line.set_data(xdata[1:], ydata[1:])
        return line, point,
    
    
    x, y, Ex, Ey = generate_vectormap(xc, yc, intenses, rect, eps=0.1)
    
    ax.quiver(x, y, Ex/np.sqrt(Ex**2+Ey**2), Ey/np.sqrt(Ex**2+Ey**2), pivot='middle', 
               headwidth=4, headlength=6, color='gray')
    ax.plot(xc, yc, 'ro')
    ani = animation.FuncAnimation(fig, run, np.arange(0, T, dt), blit=True, interval=0.001,
        repeat=False)
    plt.show()

In [8]:
xc = [-1.0, -1.0, 1.0, 1.0, 0.0]
yc = [-1.0, 1.0, -1.0, 1.0, 0.0]
intenses = [1/5, 1/5, 1/5, 1/5, 1/5, ]

#xc = [-1.0, -0.5, 1.0, 0.5, 0.0]
#yc = [0, 0, 0, 0, 0]

#Masha:
#xc = [-0.5, -0.5, 0.5, 0.5]
#yc = [-0.5, 0.5, -0.5, 0.5]
#intenses = [1/4, 1/4, 1/4, 1/4]

x0 = 0.5
y0 = 0.6

dt = 0.01
T = 100
rect = [[-2, -2],[2, 2]]

rk4_draw_path(x0, y0, dt, T, xc, yc, intenses, rect)